In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.62


In [2]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


In [3]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: AmherstWorkspace
Azure region: eastus2
Subscription id: a989cac4-671d-45c6-9d2b-ea7e9d936600
Resource group: AmherstRG


In [4]:
#import azureml._restclient.snapshots_client
#azureml._restclient.snapshots_client.SNAPSHOT_MAX_SIZE_BYTES = '1000'

In [5]:
'''
datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                      datastore_name="gpt2training",
                                                        account_name="amherstwstorageinnganzr",
                                                    container_name="gpt2training", 
                                                      account_key='<MY ACCOUNT KEY>',
                                                      create_if_not_exists=True)
'''

'\ndatastore = Datastore.register_azure_blob_container(workspace=ws, \n                                                      datastore_name="gpt2training",\n                                                        account_name="amherstwstorageinnganzr",\n                                                    container_name="gpt2training", \n                                                      account_key=\'DSmRA/zspF6JzGlu7Od4h4foh1qh/nUSi7KKeJGKfKrVLOA7e/5UdE/kGRPrLDprvmSTwG+6uuOKIhIjyejgPQ==\',\n                                                      create_if_not_exists=True)\n'

In [6]:
'''
import azureml.data
from azureml.data.azure_storage_datastore import AzureFileDatastore, AzureBlobDatastore

datastore.upload(src_dir='./pytorch-gpt2',
                 target_path='pytorch-gpt2',
                 overwrite=True,
                 show_progress=True)
'''

"\nimport azureml.data\nfrom azureml.data.azure_storage_datastore import AzureFileDatastore, AzureBlobDatastore\n\ndatastore.upload(src_dir='./pytorch-gpt2',\n                 target_path='pytorch-gpt2',\n                 overwrite=True,\n                 show_progress=True)\n"

In [7]:
print("These are your availible datastores...")
import azureml.core
from azureml.core import Workspace, Datastore
datastores = ws.datastores
for name, ds in datastores.items():
    print(name, ds.datastore_type)

These are your availible datastores...
workspaceblobstore AzureBlob
workspacefilestore AzureFile
commercial_blocks AzureBlob
machine_translation AzureBlob
commercialblockclassification AzureBlob
gpt2training AzureBlob


In [8]:
#get named datastore from current workspace
datastore = Datastore.get(ws, datastore_name='gpt2training')
ws.set_default_datastore('gpt2training')

In [9]:
datastore.as_mount()

$AZUREML_DATAREFERENCE_gpt2training

In [42]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC24r', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 1, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-12-08T17:07:54.356000+00:00', 'errors': None, 'creationTime': '2019-12-07T19:32:34.520354+00:00', 'modifiedTime': '2019-12-07T19:32:50.537541+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


In [11]:
from azureml.core import Experiment

experiment_name = 'pytorch-gpt2'
experiment = Experiment(ws, name=experiment_name)

In [43]:
import os
import shutil

project_folder = './train-on-amlcompute'
os.makedirs(project_folder, exist_ok=True)
shutil.copy('train.py', project_folder)
shutil.copy('processed_data_final.csv', project_folder)

'./train-on-amlcompute/processed_data_final.csv'

In [48]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

myenv = Environment("myenv")

myenv.docker.enabled = True
myenv.python.conda_dependencies = CondaDependencies.create(pip_packages=['utils','torch','tensorflow','azureml-sdk','argparse','pandas','numpy','transformers', 'scikit-learn'])

In [52]:
from azureml.train.dnn import PyTorch

script_params = {
    '--data_dir': datastore.path('/pytorch-gpt2').as_mount(),
    '--output_dir': './outputs'
}


estimator = PyTorch(source_directory=project_folder, 
                    script_params=script_params,
                    compute_target=compute_target,
                    entry_script='train.py',
                    use_gpu=True,
                    inputs=[datastore.path('/gpt2training').as_download(),datastore.as_mount()],
                    pip_packages=['pillow==5.4.1','git+https://github.com/huggingface/transformers'])


# Set environment
estimator.run_config.environment = myenv

WARNING - 'gpu_support' is no longer necessary; AzureML now automatically detects and uses nvidia docker extension when it is available. It will be removed in a future release.
WARNING - 'gpu_support' is no longer necessary; AzureML now automatically detects and uses nvidia docker extension when it is available. It will be removed in a future release.
WARNING - framework_version is not specified, defaulting to version 1.2.
WARNING - 'gpu_support' is no longer necessary; AzureML now automatically detects and uses nvidia docker extension when it is available. It will be removed in a future release.


In [53]:
run = experiment.submit(estimator)
print(run)

WARNING - 'gpu_support' is no longer necessary; AzureML now automatically detects and uses nvidia docker extension when it is available. It will be removed in a future release.
Submitting /Users/katiehouse/Documents/github/chatbot-with-personality/models/train-on-amlcompute directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


Run(Experiment: pytorch-gpt2,
Id: pytorch-gpt2_1575828353_e104bdef,
Type: azureml.scriptrun,
Status: Queued)


In [78]:
print(run.get_details())

{'runId': 'pytorch-gpt2_1575776290_3f40e08b', 'target': 'gpu-cluster', 'status': 'Preparing', 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '894b4c29-e814-40e4-ba65-53eb059ea028', 'azureml.git.repository_uri': 'https://github.com/katiehouse3/chatbot-with-personality.git', 'mlflow.source.git.repoURL': 'https://github.com/katiehouse3/chatbot-with-personality.git', 'azureml.git.branch': 'master', 'mlflow.source.git.branch': 'master', 'azureml.git.commit': '4e4b7f81f96a0d9eb7aa8eed80d1b4109efb8beb', 'mlflow.source.git.commit': '4e4b7f81f96a0d9eb7aa8eed80d1b4109efb8beb', 'azureml.git.dirty': 'True', 'AzureML.DerivedImageName': 'azureml/azureml_37d0328ca2737bfe8f22a331c771782c'}, 'runDefinition': {'script': 'train.py', 'arguments': ['--data_dir', '$AZUREML_DATAREFERENCE_4e83ae4cf64b47f88cb0396be01fb43b'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'gpu-cluster', 'dataReferences': {'4e83ae4cf64b47f88cb0396be01fb

In [79]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 'sdk_version': '1.0.62'})